<a href="https://colab.research.google.com/github/annechris13/Master-Thesis/blob/master/qpth_vs_cvxpylayers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# !pip install qpth
# !pip install cvxpy
!pip install cvxpylayers
# !pip install plotly

Processing /root/.cache/pip/wheels/a0/41/c1/ed7e505cbe573483e953f442039065d0b9a12f4fe11c14df88/cvxpylayers-0.1.2-cp36-none-any.whl
Processing /root/.cache/pip/wheels/d4/05/98/fe9a01845edd21cde036c9974b69b3291ba213ed4efc0af4e8/cvxpy-1.1.0a3-cp36-cp36m-linux_x86_64.whl
  Found existing installation: cvxpy 1.0.25
    Uninstalling cvxpy-1.0.25:
      Successfully uninstalled cvxpy-1.0.25


In [0]:
import numpy as np
import numpy.random as npr
import time
from qpth.qp import QPFunction
from cvxpylayers.torch.cvxpylayer import CvxpyLayer
import itertools
import torch
import cvxpy as cp
import pandas as pd
from scipy.linalg import sqrtm
from sklearn.metrics import mean_squared_error
import warnings
import sys
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [0]:
def prof_dense_qp(trial, nz, nbatch, cons, cuda=True):
    trials = []

    #cvxpylayers generate warnings when yielding inaccurate solutions
    #catch these warnings as errors to raise exception 
    warnings.filterwarnings('error')

    #set random seed
    npr.seed(trial)

    #initialize the random problem parameters
    L = npr.rand(nbatch, nz, nz)
    Q = np.matmul(L, L.transpose((0, 2, 1))) + 1e-3 * np.eye(nz, nz)
    p = npr.randn(nbatch, nz)

    if cons == 'dense':
        nineq = nz
        G = npr.randn(nbatch, nineq, nz)
        z0 = npr.randn(nbatch, nz)
        s0 = npr.rand(nbatch, nineq)
        h = np.matmul(G, np.expand_dims(z0, axis=(2))).squeeze(2) + s0
    elif cons == 'box':
        nineq = 2 * nz
        G = np.concatenate((-np.eye(nz), np.eye(nz)))
        G = np.stack([G] * nbatch)
        h = np.ones((nbatch, 2 * nz))
    else:
        raise NotImplementedError

    #initialize torch-tensors of problem parameters for qpth
    p_tch, Q_tch, G_tch, h_tch = [
                                  torch.from_numpy(x).double().requires_grad_()
                                  for x in [p, Q, G, h]]
    if cuda:
        p_tch, Q_tch, G_tch, h_tch = [x.cuda()
                                      for x in [p_tch, Q_tch, G_tch, h_tch]]
    e = torch.Tensor()
    #print(e)
    torch.cuda.synchronize()
    #torch.cuda.synchronize()
    #print("qpth")
    
    
    start = time.time()
    x = QPFunction(verbose=False, eps=1e-8, notImprovedLim=5,
                   maxIter=1000)(Q_tch, p_tch, G_tch, h_tch, e, e)
    qpsol=x
    torch.cuda.synchronize()
    t = time.time() - start
    
    trials.append({
        'nbatch':nbatch,
        'trial': trial,
        'cuda': cuda,
        'mode': 'qpth',
        'time': t,
        'solution':x})

    #initialize cvxpy parameters and variables 
    _Q_sqrt = cp.Parameter((nz, nz))
    _p = cp.Parameter((nz, 1))
    _G = cp.Parameter((nineq, nz))
    _h = cp.Parameter((nineq, 1))
    _z = cp.Variable((nz, 1))
    
    #define the constrained optimization problem for cvxpy
    obj = cp.Minimize(0.5 * cp.sum_squares(_Q_sqrt @ _z) + _p.T @ _z)
    cons = [_G @ _z <= _h]
    prob = cp.Problem(obj, cons)

    Q_sqrt = np.array([sqrtm(q) for q in Q])
    Q_sqrt_tch, p_tch, G_tch, h_tch = [
        torch.from_numpy(x).double().requires_grad_()
        for x in [Q_sqrt, p, G, h]]

    solver_args = {
        'mode': 'dense',
        'verbose': False,
        'max_iters': 1000,
        'eps': 1e-6,
        'use_indirect': False,
        'gpu':False,
        'n_jobs_forward': 12,
        'n_jobs_backward': 12
    }
    #print("cvxpy")
    solve = CvxpyLayer(prob, [_Q_sqrt, _p, _G, _h], [_z])
    
    #try solving the problem with cvxpy: if this generates an error, 
    #skip adding this solution to the list by returning None
    try:
      start = time.time()
      z, = solve(
          Q_sqrt_tch, p_tch.unsqueeze(-1), G_tch, h_tch.unsqueeze(-1),
          solver_args=solver_args
      )
      t = time.time() - start
    except:
      print("cvxpy warning/error \n\n Skip this solution\n")
      return None
    
    #return the solution in the shape corresponding to the qpth solution
    #for comparison 
    cpsol=z.view(nbatch,nz)
    
    trials.append({
        'nbatch':nbatch,
        'trial': trial,
        'cuda': cuda,
        'mode': f'cvxpylayers',
        'time': t,
        'solution':cpsol
    })

    return trials

In [0]:
def prof():
    #store the results of the trials in this list
    trials_final = []
    
    for nz, nbatch, cuda in itertools.product(
            [6], [128,256,512], [True]):    #trials for 3 different batchsizes
        print('\n--- {} vars/cons, batch size: {}, cuda: {} ---'.format(
            nz, nbatch, cuda))
        #i keeps track of the no.of successful optimization problem solved into the trials list
        i=0

        # j keeps track of the loop: inturn deciding the random number being generated
        j=0
        trials=[]
        #no.of times the batches should be run
        Try=True
        while Try:
            # print('  + Trial {}'.format(j))
            t = prof_dense_qp(j, nz, nbatch, 'dense', cuda)
            j=j+1

            #if an error occured while solving the problem, skip adding it to the list
            #and continue with the forloop 
            if t==None:
              continue
           
            #else add the solution and problem settings to the list
            trials += t
            #print(t)
            # print("mode: ",trials[i*2]["mode"], "   time: ",np.round(trials[i*2]["time"],4))
            # print("mode: ",trials[i*2+1]["mode"], "   time: ",np.round(trials[i*2+1]["time"],4))
            # error=mean_squared_error((trials[i*2]['solution'].to("cpu").detach().numpy())*100000,(trials[i*2+1]['solution'].detach().numpy())*100000)
            # print("Trial ",i," squared error:   ", error)
            i=i+1
            if(len(trials)>=40):
              Try=False
        trials_final += trials
    trials=trials_final
    #print(trials[1]["solution"].detach().numpy())
    #print(trials[0]["solution"].to("cpu").detach().numpy())
    for i in range(0,len(trials),2):
      # if (i==0):
        # print("cuda: ",trials[i]["cuda"])
      # elif (i==len(trials)/2):
        # print("cuda: ",trials[i]["cuda"])
      error=mean_squared_error((trials[i]['solution'].to("cpu").detach().numpy()),(trials[i+1]['solution'].detach().numpy()))
      error_4decimals=mean_squared_error((trials[i]['solution'].to("cpu").detach().numpy())*10000,(trials[i+1]['solution'].detach().numpy())*10000)
      # print("Trial ",trials[i]["trial"]," squared error:   ", error," squared error corrected to 4 decimal places:   ", error_4decimals)
    return trials

In [4]:
trials=prof() 


--- 6 vars/cons, batch size: 128, cuda: True ---

--------
qpth warning: Returning an inaccurate and potentially incorrect solution.

Some residual is large.
Your problem may be infeasible or difficult.

You can try using the CVXPY solver to see if your problem is feasible
and you can use the verbose option to check the convergence status of
our solver while increasing the number of iterations.

Advanced users:
You can also try to enable iterative refinement in the solver:
https://github.com/locuslab/qpth/issues/6
--------


--------
qpth warning: Returning an inaccurate and potentially incorrect solution.

Some residual is large.
Your problem may be infeasible or difficult.

You can try using the CVXPY solver to see if your problem is feasible
and you can use the verbose option to check the convergence status of
our solver while increasing the number of iterations.

Advanced users:
You can also try to enable iterative refinement in the solver:
https://github.com/locuslab/qpth/issues/

In [0]:
results=pd.DataFrame(columns=["index","trial_no","qpth_time","cvxpylayers_time","nbatch","error","error-4decimal places"])

for i in range(0,len(trials),2):
  result=[]
  idx=i/2
  trial=trials[i]['trial']
  qpth_time=trials[i]['time']
  cvxpylayers_time=trials[i+1]['time']
  error=mean_squared_error((trials[i]['solution'].to("cpu").detach().numpy()),(trials[i+1]['solution'].detach().numpy()))
  error_4=mean_squared_error((trials[i]['solution'].to("cpu").detach().numpy())*10000,(trials[i+1]['solution'].detach().numpy())*10000)

  nbatch=trials[i]['nbatch']
  result=pd.Series({'index':idx,'trial_no':trial,'qpth_time':qpth_time,'cvxpylayers_time':cvxpylayers_time,'nbatch':nbatch,'error':np.round(error),'error-4decimal places':np.round(error_4)})
  results=results.append(result,ignore_index=True)

  #print("Trial ",trials[i]['solution']," squared error:   ", error)


In [0]:
# results
results_grouped_time = results.groupby("nbatch").mean().reset_index()
results_grouped_time['nbatch']=results_grouped_time['nbatch'].astype(str)
# results_grouped_time

In [0]:
nb=['n= ']+results_grouped_time["nbatch"].astype(str)
qp=results_grouped_time['qpth_time']
cpl=results_grouped_time['cvxpylayers_time']
results['error-4bool']=(results['error-4decimal places']>0).astype(int)
results['error-bool']=(results['error']>0).astype(int)
# results

In [8]:
plt=go.Figure()
plt.add_trace(go.Bar(x=nb ,y=qp,name="qpth GPU"))#,row=1,col=1)
plt.add_trace(go.Bar(x=nb ,y=cpl,name="cvxpylayers "))#,row=1,col=1)
plt.update_yaxes(title="Average Time in Seconds")#,row=1,col=1)
plt.update_xaxes(title="Batch Size")#,row=1,col=1)
plt.update_layout(width=900,height=400)
plt.show()

plt=go.Figure()
plt.add_trace(go.Scatter(x=results["index"] ,y=results["qpth_time"],name="qpth GPU - Time",mode="lines+markers",marker_color="blue"))#,row=2,col=1)
plt.add_trace(go.Scatter(x=results["index"] ,y=results["cvxpylayers_time"],name="cvxpylayers - Time",mode="lines+markers",marker_color="red"))#,row=2,col=1)
plt.add_trace(go.Bar(x=results["index"] ,y=results["error-bool"]*5,name="error",marker_color="red",opacity=0.3))#,row=2,col=1)
plt.add_trace(go.Bar(x=results["index"] ,y=results["error-4bool"]*5,name="error (precision of 4 decimal places)",marker_color="red",opacity=0.5))#,row=2,col=1)
plt.update_layout(width=1200,height=400)
plt.update_xaxes(title="20 observations each with nBatch in [128,256,512]")
plt.show()
# plt.add_trace(go.Scatter(x=nb.astype(str) ,y=cpl,name="cvxpylayers "),row=1,col=1)
# plt.update_yaxes(title="Average Time in Seconds",row=1,col=1)

